# Importing necessary libraries, setting up document

https://course.fast.ai/videos/?lesson=1

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
# The notebook reloads automatically, have plots stored in document

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate

import pandas as pd # Data processing
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path
# Import some libraries to use

<font size="5">**Preparing data for the model**

In [ ]:
data_folder = Path("../input/aptos2019-blindness-detection")
data_folder.ls()
# Allows for the manipulation of folders, see file structure used

In [ ]:
train_df = pd.read_csv(data_folder/'train.csv')
test_df = pd.read_csv(data_folder/'test.csv')
# Read data into the dataframes for training and testing, allows for easy manipulation of the data

https://blog.usejournal.com/finding-data-block-nirvana-a-journey-through-the-fastai-data-block-api-c38210537fe4
https://docs.fast.ai/data_block.html

In [ ]:
test_data = ImageList.from_df(test_df, path=data_folder, folder='test_images', suffix='.png')
# Object to represent test set, using data block API

In [ ]:
bs = 64
#Set batch size

In [ ]:
# Data block API, prepares data for training, use in the model 
data = (ImageList.from_df(train_df, path=data_folder, folder='train_images', suffix = '.png')
                 .split_by_rand_pct(valid_pct=0.2) #split between training, validation sets
                 .label_from_df()
                 .add_test(test_data)              
                 .transform(get_transforms(flip_vert=True), size=224)
                 .databunch(path='.', bs=bs) #use bs from earlier
                 .normalize(imagenet_stats)
       )

In [ ]:
data.show_batch(rows=3, figsize=(10,10))
# See some of the images we are working with

# Training, running the model

In [ ]:
# Create the learner for a convolutional neural network
learn = cnn_learner(data, models.resnet34, pretrained=False, model_dir = "/output/kaggle/working", metrics=error_rate)

https://www.kaggle.com/tanlikesmath/oversampling-mnist-with-fastai

https://docs.fast.ai/callbacks.lr_finder.html#LRFinder

In [ ]:
# Find optimal learning rate to use for the model
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
min_grad_lr = learn.recorder.min_grad_lr
min_grad_lr

In [ ]:
learn.fit_one_cycle(10, min_grad_lr)

In [ ]:
#Retrain model and fine-tune the learning rate
learn.save('initial') # Set of weights created with learn, saves the model for possible future use/access
learn.unfreeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
# Fine-tuning, new learning rate
min_grad_lr2 = learn.recorder.min_grad_lr
min_grad_lr2

# Results

In [ ]:
learn.fit_one_cycle(10, min_grad_lr2)
learn.save('final')